In [69]:
import random
import sys
sys.path.append('..')

import dataloader
import utils
import figures

import pickle5 as pickle
import numpy as np
import pandas as pd

In [7]:
samples, sample_genes, sample_barcodes = dataloader.load_processed_spots()
positions = dataloader.load_positions(sample_barcodes, positions_path='../data/spatial/tissue_positions_list.csv')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.var` of view, copying.


In [34]:
samples.shape, len(sample_genes), len(sample_barcodes)

((2896, 30), 30, 2896)

In [108]:
# get times
timedict = pickle.load(open('../data/pseudo_exprdist.pickle', 'rb+'))
times = np.array(list(timedict.keys()))
time_barcodes = np.array(list(timedict.values()))
idx = np.argsort(times)
times = times[idx]
time_barcodes = time_barcodes[idx]

In [109]:
train_idx = np.zeros(len(times))
train_idx[::2] = 1
train_idx = train_idx.astype(bool)

In [110]:
def make_df(barcode_timeseries):
    time_idx = [sample_barcodes.index(barcode) for barcode in barcode_timeseries]
    df = pd.DataFrame(samples[time_idx], columns=sample_genes)
    return df

train_df = make_df(time_barcodes[train_idx])
test_df = make_df(time_barcodes[~train_idx])

In [111]:
lags = 2
W_dy = utils.learn_dbn(train_df, lags)

In [112]:
true_mse = utils.mse(W_dy, test_df, lags)
true_mse

0.1280684980434823

In [113]:
# test significance
n = 25
mses = np.zeros(n)
for i in range(n):
    print(i)
    train_barcodes = time_barcodes[train_idx]
    random.shuffle(train_barcodes)
    random_df = make_df(train_barcodes)
    random_W = utils.learn_dbn(random_df, lags)
    random_mse = utils.mse(random_W, test_df, lags)
    mses[i] = random_mse
true_mse, mses

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


(0.1280684980434823,
 array([0.13301361, 0.13252257, 0.13281059, 0.13291727, 0.13356909,
        0.1328262 , 0.13371927, 0.13397408, 0.1333559 , 0.13266774,
        0.13306556, 0.13346569, 0.1334188 , 0.13341722, 0.13277946,
        0.13291347, 0.13266345, 0.13288173, 0.13177952, 0.13274451,
        0.13372814, 0.13219865, 0.1331796 , 0.13327074, 0.13301667]))

In [114]:
(true_mse - np.mean(mses))/ np.std(mses)

-10.24030910278509

In [115]:
true_mse < mses

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])